# Create Sun Cloud traffic analysis zone layer.

Define default workspace and data source.

In [2]:
import arcpy
import pandas as pd

arcpy.env.overwriteOutput = True
df = pd.read_excel("./data/data_dictionary.xlsx")

db = arcpy.env.workspace = "./data/sun_cloud_taz.gdb"

arcpy.Describe(db)
orig_data = 'SunCloud_TAZ_2019_2021_2050_Combined'
target_sr = arcpy.SpatialReference(2223)

Make count fields interger field.

In [ ]:
fc='sun_cloud_taz'
arcpy.management.CreateFeatureclass(
    out_path=db,
    out_name= fc,
    geometry_type="POLYGON",
    template=orig_data,
    spatial_reference=target_sr
)

In [ ]:
# delete out all number fields
fc='sun_cloud_taz'
number_fields=[i.name for i in arcpy.ListFields(fc) if i.type=='Double' and i.required == False]
number_fields

arcpy.DeleteField_management(fc, number_fields)

In [ ]:
for field in number_fields:
    arcpy.AddField_management(fc, field, 'LONG')

In [ ]:
# load data
arcpy.management.Append(
    inputs="SunCloud_TAZ_2019_2021_2050_Combined",
    target="\sun_cloud_taz",
    schema_type="NO_TEST"
)

Convert the field aliases using the data dictionary.

In [ ]:
# df = df.reset_index()  # make sure indexes pair with number of rows
fc = 'sun_cloud_taz'
for index, row in df.iterrows():
    name = row['Field Name'].rstrip()
    print(name)
    alias = (row['Description']).rstrip()
    print(alias)
    arcpy.management.AlterField(in_table=fc, field=name, new_field_alias=alias)

In [ ]:
fields=[(i.name, i.aliasName) for i in arcpy.ListFields(fc)]
fields

Convert field name to lower case.

In [ ]:
fc='sun_cloud_taz'
fields=[i.name for i in arcpy.ListFields(fc) if i.required == False]
fields

In [ ]:
for f in fields:
    print(f)
    arcpy.management.AlterField(in_table=fc, field=f, new_field_name="_{}".format(f))

In [ ]:
fc='sun_cloud_taz'
fields=[i.name for i in arcpy.ListFields(fc) if i.required == False]
fields

In [ ]:
for f in fields:
    if(f[0]=='_'):
        new_name = (f[1:]).lower()
        print(new_name)
        arcpy.management.AlterField(in_table=fc, field=f, new_field_name=new_name)

In [ ]:
fields=[i.name for i in arcpy.ListFields(fc) if i.required == False]
fields

Add new fields and calculate values. 

In [5]:
new_fields = [
    ('primary_emp_sector_2019', 'Primary Employment Sector (2019)', 'TEXT'),
	('emp_change_2019_2050', 'Employment Change (2019 to 2050)', 'LONG'),
    ('hh_change_2019_2050', 'Households Change (2019 to 2050)', 'LONG'),
	('pop_change_2019_2050', 'Population Change (2019 to 2050)', 'LONG'),
	('emp_density_2019', 'Jobs per Square Mile (2019)', 'LONG'),
	('hh_density_2019', 'Households per Square Mile (2019)', 'LONG'),
    ('pop_density_2019', 'Population per Square Mile (2019)', 'LONG')]
for (field, alias, type) in new_fields:
    arcpy.AddField_management(in_table=fc, field_name=field, field_alias=alias, field_type=type)


Attribute the new fields.

In [6]:
fc ='sun_cloud_taz'
arcpy.management.CalculateField(
    in_table=fc,
    field='emp_change_2019_2050',
    expression="int(!totemp_2050! - !totemp_2019!)",
    expression_type="PYTHON3",

    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result './data/sun_cloud_taz.gdb\\sun_cloud_taz'>

In [66]:
fc ='sun_cloud_taz'
arcpy.management.CalculateField(
    in_table=fc,
    field='hh_change_2019_2050',
    expression="int(!tothh_2050! - !tothh_2019!)",
    expression_type="PYTHON3",

    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result './data/sun_cloud_taz.gdb\\sun_cloud_taz'>

In [67]:
arcpy.management.CalculateField(
    in_table=fc,
    field='pop_change_2019_2050',
    expression="int(!totpop_2050! - !totpop_2019!)",
    expression_type="PYTHON3",

    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result './data/sun_cloud_taz.gdb\\sun_cloud_taz'>

In [63]:
# emp density
# 27,878,400
fc='sun_cloud_taz'
arcpy.management.CalculateField(
    in_table=fc,
    field='emp_density_2019',
    expression="!totemp_2019!/(!Shape_Area!/27878400)",
    expression_type="PYTHON3",

    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateField(
    in_table=fc,
    field='pop_density_2019',
    expression="!totpop_2019!/(!Shape_Area!/27878400)",
    expression_type="PYTHON3",

    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateField(
    in_table=fc,
    field='hh_density_2019',
    expression="!tothh_2019!/(!Shape_Area!/27878400)",
    expression_type="PYTHON3",
)

<Result './data/sun_cloud_taz.gdb\\sun_cloud_taz'>

In [61]:
fields_desc = {
'primary_emp_sector_2019':'',
'public_emp_2019':'Public sector',
'retail_emp_2019':'Retail',
'ind_emp_2019':'Industrial',
'other_emp_2019':'Other',
'nsb_emp_2019':'Non-site based employment',
'con_emp_2019':'Construction',
'whlsl_emp_2019':'Wholesale',
'fire_emp_2019':'Finance, insurance, real-estate',
'serv_emp_2019':'Service sector',
'hwyret_2019':'Highway retail',
'workhome_2019':'Work at home',
'ag_emp_2019':'Agricultural',
'health_emp_2019':'Healthcare',
'leisure_emp_2019':'Leisure',
'mining_emp_2019':'Mining',
}

fields = [item for item in fields_desc]

with arcpy.da.UpdateCursor(fc, fields) as cur:
    for row in cur:
        candidates = list(row[1:])
        # print(candidates)
        selected = fields[candidates.index(max(candidates))+1]
        value = fields_desc[selected]
        row[0] = value
        cur.updateRow(row)
